In [1]:
# Kaggle setup: install ultralytics + dependencies
!pip install ultralytics==8.2.90 torch torchvision opencv-python timm thop einops grad-cam dill


In [2]:
import types, sys

module_name = "spd_ema_modules"

code = r'''
import torch
import torch.nn as nn
import torch.nn.functional as F

class SPDConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1,
                 bias=False, use_modulation=False):
        super().__init__()
        self.use_modulation = use_modulation
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride,
                              padding, bias=bias)
        if use_modulation:
            self.scale = nn.Parameter(torch.ones(1, out_channels, 1, 1))
            self.shift = nn.Parameter(torch.zeros(1, out_channels, 1, 1))

    def forward(self, x):
        out = self.conv(x)
        if self.use_modulation:
            out = out * self.scale + self.shift
        return out

class EMAAttention(nn.Module):
    def __init__(self, channels, decay=0.999):
        super().__init__()
        self.decay = decay
        self.mu = nn.Parameter(torch.zeros(1, channels, 1, 1), requires_grad=False)
        self.gamma = nn.Parameter(torch.ones(1, channels, 1, 1))

    def forward(self, x):
        with torch.no_grad():
            self.mu.mul_(self.decay).add_(x.mean(dim=(0, 2, 3), keepdim=True) * (1 - self.decay))
        return x * self.gamma + self.mu

def replace_conv_with_spd(model, use_modulation=True):
    for name, module in list(model.named_children()):
        if isinstance(module, nn.Conv2d):
            new_conv = SPDConv(
                module.in_channels, module.out_channels,
                module.kernel_size[0], module.stride[0],
                module.padding[0], bias=(module.bias is not None),
                use_modulation=use_modulation
            )
            setattr(model, name, new_conv)
        else:
            replace_conv_with_spd(module, use_modulation)
'''

mod = types.ModuleType(module_name)
exec(code, mod.__dict__)
sys.modules[module_name] = mod

import spd_ema_modules as se
print("✅ In-memory module created:", se)
print("Classes loaded:", [n for n in dir(se) if not n.startswith("_")])


✅ In-memory module created: <module 'spd_ema_modules'>
Classes loaded: ['EMAAttention', 'F', 'SPDConv', 'nn', 'replace_conv_with_spd', 'torch']


In [3]:
from ultralytics import YOLO
import spd_ema_modules as se
import torch.nn as nn

# Load pretrained YOLOv8 model (nano for testing)
model = YOLO("yolov8n.pt")

# Replace Conv layers with SPDConv
se.replace_conv_with_spd(model.model, use_modulation=True)

# Add EMAAttention to an intermediate stage (example: layer index 2)
model.model.model[2] = nn.Sequential(model.model.model[2], se.EMAAttention(64))

print("✅ SPDConv + EMA integrated successfully!")


✅ SPDConv + EMA integrated successfully!


In [4]:
# --- FINAL FIX: Disable Ray callbacks in Ultralytics ---
import sys, types

# Fake a minimal ray module so Ultralytics doesn't crash
ray_stub = types.SimpleNamespace(train=types.SimpleNamespace(_internal=types.SimpleNamespace(session=None)))
sys.modules['ray'] = ray_stub

# Patch the RayTune callback inside Ultralytics to do nothing
import ultralytics.utils.callbacks.raytune as raytune

def disable_ray_callback(*args, **kwargs):
    return None

raytune.on_fit_epoch_end = disable_ray_callback
raytune.on_train_start = disable_ray_callback
raytune.on_train_end = disable_ray_callback
raytune.on_train_epoch_end = disable_ray_callback

print("✅ RayTune callbacks fully disabled — safe to train now.")


✅ RayTune callbacks fully disabled — safe to train now.


In [5]:
from ultralytics import YOLO
import spd_ema_modules as se
import torch.nn as nn
import os

# --- Disable WandB + Clear its callbacks ---
os.environ["WANDB_MODE"] = "disabled"
import ultralytics
ultralytics.utils.callbacks.wb = {}   # ✅ Disable WandB callback hooks

# --- Paths ---
data_yaml = "/kaggle/input/helmet-detection/data.yaml"  # <-- change this
model = YOLO("yolov8n.pt")

# --- Inject SPDConv + EMA Attention ---
se.replace_conv_with_spd(model.model, use_modulation=True)
model.model.model[2] = nn.Sequential(model.model.model[2], se.EMAAttention(64))

# --- Train the modified model ---
results = model.train(
    data=data_yaml,
    epochs=10,
    imgsz=640,
    batch=16,
    project="spd_ema_runs",  # ✅ no slashes
    name="train",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.90 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/helmet-detection/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=spd_ema_runs, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

E0000 00:00:1761747485.739082     430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761747485.746364     430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: Unsupp

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/helmet-detection/train/labels... 22575 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22575/22575 [00:39<00:00, 575.89it/s]


train: WARNING ⚠️ Cache directory /kaggle/input/helmet-detection/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


10 epochs completed in 0.660 hours.
Optimizer stripped from spd_ema_runs/train/weights/last.pt, 6.2MB
Optimizer stripped from spd_ema_runs/train/weights/best.pt, 6.2MB

Validating spd_ema_runs/train/weights/best.pt...
Ultralytics YOLOv8.2.90 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:07<00:00,  4.04it/s]


                   all       1000       5112      0.938      0.561      0.615      0.374
                  head        174       1063      0.897      0.827      0.897      0.544
                helmet        914       3911      0.918      0.856      0.927      0.569
                person         30        138          1          0     0.0201    0.00769


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to spd_ema_runs/train


In [8]:
# === RUN INFERENCE ON TEST IMAGES ===
best_model_path = "/kaggle/working/spd_ema_runs/train/weights/best.pt"

model = YOLO(best_model_path)
test_dir = "/kaggle/input/helmet-detection/test/images"  # <-- change this path

model.predict(
    source=test_dir,
    save=True,
    conf=0.25,
    project="/kaggle/working/runs",
    name="spd_ema_predictions",
    exist_ok=True
)



image 1/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers1034_png.rf.547306eb43c66046d020dda3a108d585.jpg: 640x640 11 heads, 7.2ms
image 2/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers106_png.rf.310eb4163018baec8d559f135fc1c911.jpg: 640x640 2 helmets, 7.2ms
image 3/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers107_png.rf.2ca4b25a457b442b8fe309b3a0966e05.jpg: 640x640 1 helmet, 7.2ms
image 4/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers1085_png.rf.0f451da4103d8d4e20cf7d58dfe47884.jpg: 640x640 2 helmets, 7.2ms
image 5/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers1090_png.rf.87a98780398676e8fb53244db82ad858.jpg: 640x640 3 helmets, 7.2ms
image 6/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers1109_png.rf.22b4da5b952a2cb5749cbea814b25f9a.jpg: 640x640 6 helmets, 7.2ms
image 7/500 /kaggle/input/helmet-detection/test/images/hard_hat_workers1118_png.rf.e46abf769a24c0c9fa082719abe70f6a.jpg: 640x64

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'head', 1: 'helmet', 2: 'person'}
 obb: None
 orig_img: array([[[208, 208, 208],
         [208, 208, 208],
         [208, 208, 208],
         ...,
         [182, 172, 172],
         [184, 174, 174],
         [185, 175, 175]],
 
        [[208, 208, 208],
         [208, 208, 208],
         [208, 208, 208],
         ...,
         [185, 175, 175],
         [186, 176, 176],
         [187, 177, 177]],
 
        [[208, 208, 208],
         [208, 208, 208],
         [208, 208, 208],
         ...,
         [190, 180, 180],
         [190, 180, 180],
         [190, 180, 180]],
 
        ...,
 
        [[ 91, 139, 187],
         [ 94, 142, 190],
         [ 97, 146, 194],
         ...,
         [139, 180, 219],
         [146, 187, 226],
         [150, 191, 230]],
 
        [[ 91, 140, 186],
         [ 95, 144, 190],
         [ 98, 147, 195],
        

In [9]:
import shutil

output_dir = "/kaggle/working/runs/spd_ema_predictions"
zip_path = "/kaggle/working/spd_ema_test_outputs.zip"

shutil.make_archive(zip_path.replace(".zip", ""), "zip", output_dir)
print("✅ Zipped results at:", zip_path)


✅ Zipped results at: /kaggle/working/spd_ema_test_outputs.zip


In [10]:
import shutil
import os

# Path where YOLOv8 results are stored
results_dir = "/kaggle/working/spd_ema_runs/train"
zip_path = "/kaggle/working/spd_ema_runs/yolo_results.zip"

# Remove existing zip if any
if os.path.exists(zip_path):
    os.remove(zip_path)

# Create a zip of all result files (curves, confusion matrix, etc.)
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', results_dir)

print("✅ All YOLOv8 result files have been zipped successfully!")
print("📦 Zip location:", zip_path)


✅ All YOLOv8 result files have been zipped successfully!
📦 Zip location: /kaggle/working/spd_ema_runs/yolo_results.zip
